In [43]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install scipy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
from datasets import load_dataset

dataset_name = 'Ali-C137/Arabic_guanaco_oasst1' 
dataset = load_dataset(dataset_name, split="train")

Found cached dataset parquet (/home/studio-lab-user/.cache/huggingface/datasets/Ali-C137___parquet/Ali-C137--Arabic_guanaco_oasst1-4294f765deb60d5a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [66]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [67]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [68]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [69]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [70]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/Ali-C137___parquet/Ali-C137--Arabic_guanaco_oasst1-4294f765deb60d5a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-12dd73602ef012c1.arrow


In [71]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [72]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.289700
20,1.194400
30,1.150100
40,1.145400
50,1.137200
60,1.140200
70,1.123100
80,1.074200
90,1.105700
100,1.076700


TrainOutput(global_step=100, training_loss=1.1436949348449708, metrics={'train_runtime': 2112.1819, 'train_samples_per_second': 0.758, 'train_steps_per_second': 0.047, 'total_flos': 1.477539887628288e+16, 'train_loss': 1.1436949348449708, 'epoch': 0.16})

In [73]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [79]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [82]:
dataset['text']

['### الإنسان: هل يمكنك كتابة مقدمة قصيرة حول أهمية مصطلح "احتكار الشراء" في الاقتصاد؟ الرجاء استخدام الأمثلة المتعلقة بعمليات احتكار الشراء المحتملة في سوق العمل والاستشهاد بالبحث ذي الصلة. ### المساعد: يشير مصطلح "Monopsony" إلى هيكل السوق حيث يوجد مشتر واحد فقط لسلعة أو خدمة معينة. في الاقتصاد ، يعتبر هذا المصطلح وثيق الصلة بشكل خاص بسوق العمل ، حيث يتمتع صاحب العمل الاحتكاري بسلطة كبيرة على أجور وظروف عمل موظفيه. يمكن أن يؤدي وجود احتكار الشراء إلى انخفاض الأجور وتقليل فرص العمل للعمال ، حيث أن صاحب العمل لديه حافز ضئيل لزيادة الأجور أو توفير ظروف عمل أفضل.\n\nحددت الأبحاث الحديثة عمليات احتكار الشراء المحتملة في صناعات مثل البيع بالتجزئة والوجبات السريعة ، حيث يتحكم عدد قليل من الشركات الكبيرة في جزء كبير من السوق (Bivens & Mishel ، 2013). في هذه الصناعات ، غالبًا ما يواجه العمال أجورًا منخفضة ومزايا محدودة وقوة تفاوضية منخفضة ، مما يؤدي إلى وضع يعتمدون فيه على صاحب العمل في معيشتهم. يمكن أن يؤدي هذا الاعتماد إلى مزيد من قمع الأجور وتدهور في ظروف العمل.\n\nبشكل عام ، يعد مفهوم احت

In [85]:
text = "لإنسان: سرد ووصف العناصر المميزة للعمارة اليونانية في العصور الكلاسيكية ### المساعد: تتميز العمارة اليونانية في العصور الكلاسيكية بالبحث عن "
device = "cuda"

In [86]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None 

لإنسان: سرد ووصف العناصر المميزة للعمارة اليونانية في العصور الكلاسيكية ### المساعد: تتميز العمارة اليونانية في العصور الكلاسيكية بالبحث عن مواقف متنوعة ومتعاونة ومتميزة بالمعاني المتعاقدة بالمع


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [91]:
from huggingface_hub import login
login()